In [1]:
%pip install semantic-link-labs
%pip install semantic-link-sempy

StatementMeta(, 0524ed55-f82f-4ef3-bd7f-aab7a33d81c6, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.3/699.3 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 116.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.9.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-3294f8d7-2de8-41f9-9f82-5704d0c38ee4
    Can't uninstall 'semantic-link-sempy'. No files were found to uninstall.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, current_date
from delta.tables import DeltaTable
import json
import pandas as pd
import requests
import concurrent.futures
import time
import msal
import datetime

StatementMeta(, 0524ed55-f82f-4ef3-bd7f-aab7a33d81c6, 10, Finished, Available, Finished)

In [3]:
import sempy_labs as labs
from sempy_labs import migration, directlake, admin, graph, vertipaq_analyzer
from sempy_labs import lakehouse as lake
from sempy_labs import report as rep
from sempy_labs.tom import connect_semantic_model
from sempy_labs.report import ReportWrapper
import sempy
sempy.fabric._client._utils._init_analysis_services()
import Microsoft.AnalysisServices.Tabular as TOM


StatementMeta(, 0524ed55-f82f-4ef3-bd7f-aab7a33d81c6, 11, Finished, Available, Finished)

In [4]:
import sempy.fabric as fabric
import pandas as pd
import datetime


from sempy_labs._helper_functions import (
    resolve_lakehouse_name,
    save_as_delta_table,
    resolve_workspace_capacity,
    retry,
    _get_column_aggregate,
    resolve_workspace_id,
    resolve_lakehouse_name_and_id,
)
from sempy_labs.lakehouse import (
    get_lakehouse_tables,
    lakehouse_attached,
)

from sempy_labs._model_bpa import run_model_bpa
from typing import Optional, List
from sempy._utils._log import log
import sempy_labs._icons as icons
from uuid import UUID

StatementMeta(, 0524ed55-f82f-4ef3-bd7f-aab7a33d81c6, 12, Finished, Available, Finished)

In [14]:
@log
def run_power_insighter(
    language: Optional[str] = None,
    workspace: Optional[str | UUID | List[str | UUID]] = None,
    skip_models: Optional[str | List[str]] = ["ModelBPA", "Fabric Capacity Metrics"],
    skip_models_in_workspace: Optional[dict] = None,
):
    """
    Runs all the sempy functions and then it will create a Semantic Model where you will get all the articats and directly you can do reporting

    Parameters
    ----------
    language : str, default=None
        The language (code) in which the rules will appear. For example, specifying 'it-IT' will show the Rule Name, Category and Description in Italian.
        Defaults to None which resolves to English.
    workspace : str | uuid.UUID | List[str | uuid.UUID], default=None
        The workspace or list of workspaces to scan. Supports both the workspace name and the workspace id.
        Defaults to None which scans all accessible workspaces.
    skip_models : str | List[str], default=['ModelBPA', 'Fabric Capacity Metrics']
        The semantic models to always skip when running this analysis.
    skip_models_in_workspace : dict, default=None
        A dictionary showing specific semantic models within specific workspaces to skip. See the example below:
        {
            "Workspace A": ["Dataset1", "Dataset2"],
            "Workspace B": ["Dataset5", "Dataset 8"],
        }
    """
    if not lakehouse_attached():
        raise ValueError(
            f"{icons.red_dot} No lakehouse is attached to this notebook. Must attach a lakehouse to the notebook."
        )

    if isinstance(skip_models, str):
        skip_models = [skip_models]

    skip_models.extend(["ModelBPA", "Fabric Capacity Metrics"])

    now = datetime.datetime.now()

    #Pulling all the workspaces details
    dfW = fabric.list_workspaces("type ne 'AdminInsights'")
    if workspace is None:
        dfW_filt = dfW.copy()
    else:
        dfW_filt = dfW[(dfW["Name"].isin(workspace)) | (dfW["Id"].isin(workspace))]

    if dfW_filt.empty:
        raise ValueError(
            f"{icons.red_dot} There are no valid workspaces to assess. This is likely due to not having proper permissions to the workspace(s) entered in the 'workspace' parameter."
        )

    # Convert 'Name' column to list
    workspaces_list = dfW_filt['Id'].tolist()



    all_dfD = pd.DataFrame()
    all_DatasetSourceDetails = pd.DataFrame()
    all_dfApps = pd.DataFrame()
    all_dfDataFlows = pd.DataFrame()

    # all_dfCapacities = fabric.list_capacities()
    for _, r in dfW_filt.iterrows():
        wksp = r["Name"]
        wksp_id = r["Id"]
        # capacity_id, capacity_name = resolve_workspace_capacity(workspace=wksp)

        # df = pd.DataFrame(columns=list(icons.bpa_schema.keys()))

        # Get Dataset details of a workspace
        dfD = fabric.list_datasets(workspace=wksp, mode="rest")

        #Adding Workspace ID to the dataframe
        dfD["Workspace Id"] = wksp_id

        #Creating Datasets List
        Datasets_list = dfD['Dataset Id'].tolist()

        # Append to the master DataFrame
        all_dfD = pd.concat([all_dfD, dfD], ignore_index=True)

        #Pulling all apps 
        all_dfApps = fabric.list_apps()

        #Pulling all capacities
        

        #List DataFlows
        all_dfDataFlows = fabric.list_dataflows()

    return dfD

StatementMeta(, 0524ed55-f82f-4ef3-bd7f-aab7a33d81c6, 22, Finished, Available, Finished)

In [8]:
test = run_power_insighter()

StatementMeta(, 0524ed55-f82f-4ef3-bd7f-aab7a33d81c6, 16, Finished, Available, Finished)

In [12]:
t = test['Dataset Id'].tolist()

StatementMeta(, 0524ed55-f82f-4ef3-bd7f-aab7a33d81c6, 20, Finished, Available, Finished)

In [13]:
print(t)

StatementMeta(, 0524ed55-f82f-4ef3-bd7f-aab7a33d81c6, 21, Finished, Available, Finished)

['b77c9691-f02f-4c6d-952d-71ef01fdca44', 'ee8ea3bb-849f-40c5-998b-c4ab4cdd8903', '2229ba56-cec0-4c46-b343-22a3a1b89428', 'f9880865-226c-42b7-9eab-19a93704a3b6', '2277ef0e-e77d-459b-becc-4d18acb0e66b', '4ece083e-f1a0-491d-b0cd-5e44c2ded4b7', '503a79b1-12e2-4451-b161-556f15e07ec0', 'a0533b96-4d0b-4635-b227-2a1435b29aac', 'db717a69-0e64-42ae-9fbc-028a15f262a5', '654f7438-1687-4485-8c03-653165caac19', '027aeb2b-b88c-4043-a791-72d411a36bc4', 'd5e8df44-a91e-4d2a-a9e4-3cae1884e4f7', '241e13f0-7e00-4b62-bbb0-54a0eb8d18d7', 'f274bb64-19bd-4687-8605-4523db3bcb4b', 'ad322ede-6861-4552-a329-babcb1662ac9', '2619faf8-1006-4883-b60c-b3be3e1a2ff8', '074a8b85-03f1-4ae2-ab20-953e2045fae6', 'd356f8f6-d041-43be-9ab4-24939568048b', '907d0025-0f0d-4d85-a17a-9c259ed4b849', 'ca5907f3-4cd7-4db8-aa30-18796ecc5de2', 'a6875075-a9c9-41bc-9bf7-c66b3132e3ea', '37dcd7ae-3629-44da-af62-81a85fbff8c3', 'ec4bccff-7c3a-4439-b160-4e308ea924d1', '6694f654-1478-4934-9182-3484e799642e', '73e7dd46-0cd3-490a-a5f6-df181537df10',

In [12]:
test

StatementMeta(, c7b6947b-e5ba-4903-9b7b-cc2e0919af09, 26, Finished, Available, Finished)

,Id,Is Read Only,Is On Dedicated Capacity,Capacity Id,Default Dataset Storage Format,Type,Name
0,35060111-ead1-48c7-90ad-6470f10fc59e,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,3. Domestic 2W Marketing
1,fd2fc524-9196-41e8-9ecb-2f4e8e8ded50,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,1. Domestic 2W Sales
2,e86870f9-b5ef-4562-824c-4fb553d63e7b,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,5. Domestic 2W Digital Assets
3,83ec31a1-2ca3-46fe-a5f8-2bcc8e70f8cd,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,2. Domestic 2W Dealer Management
4,13a5a575-b5bb-4e9f-8e12-499d4af44889,False,False,NaN,NaN,Workspace,Development_Workspace
5,7acb5d88-77de-4762-bb6d-57b1ce0afa46,False,False,NaN,NaN,Workspace,BI QC
6,228db8e8-6203-4ba8-b406-7b1da29c34c1,False,True,76f4499e-05ff-44a9-8c2f-323ee14ea1a7,Small,Workspace,P1 Priority Datasets
7,a912e8a5-7138-4452-88a1-5c7df2c99a5a,False,False,NaN,NaN,Workspace,Beta Testing Workspace
8,8b09a058-33d3-4ef3-a8dd-f4bc5e740652,False,True,22b9217a-6e1d-4f0d-ac84-cd31903c88a2,Small,Workspace,7. International Business
9,8ba77fd7-a1ef-4837-8f15-915ba50d0f4a,False,True,517b62f7-2477-4809-a9d5-bfbec118442c,Small,Workspace,BI Governance
